In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing # for label encoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

### Read the data
We read the data from a local csv file and construct our matrix of feature vectors

In [106]:
data_orig = pd.read_csv("data\play_by_play_2019.csv", dtype=object) # maybe should be dtype=str
# Trim up our data to only features we care about
feature_vectors = data_orig[["touchdown", "home_team", "away_team", "season_type", "yardline_100", "quarter_seconds_remaining", "half_seconds_remaining",
                 "game_seconds_remaining", "game_half", "drive", "sp", "qtr", "down", "goal_to_go", "ydstogo", "play_type", "rush_attempt", "pass_attempt" ]]
# Once I get to a certain point, I should try adding data from other csv files to this dataframe as well

In [107]:
# In this cell I should add ELO ratings to each row in the data.  
# Going to skip that for now and just do this with data we already have

### Clean the data
Need to clean up the data so it can be processed by the KNN algorithm

In [108]:
#Delete all columns that have bad plays (i.e. kickoffs, field goals)
feature_vectors = feature_vectors[feature_vectors.yardline_100.notnull()]
feature_vectors = feature_vectors[feature_vectors.down.notnull()]
feature_vectors = feature_vectors[feature_vectors.play_type.notnull()]

# Go through and fill in missing values for the columns that initially had NA values
#feature_vectors["down"] = feature_vectors["down"].fillna(-1)
#feature_vectors["play_type"] = feature_vectors["play_type"].fillna("yikes")
#feature_vectors["rush_attempt"] = feature_vectors["rush_attempt"].fillna(-1)
#feature_vectors["pass_attempt"] = feature_vectors["pass_attempt"].fillna(-1)
#print(feature_vectors["play_type"])

# Encode columns in feature_vectors so that there aren't any strings 
le = preprocessing.LabelEncoder()
feature_vectors["home_team"] = le.fit_transform(feature_vectors["home_team"])
feature_vectors["away_team"] = le.fit_transform(feature_vectors["away_team"])
feature_vectors["season_type"] = le.fit_transform(feature_vectors["season_type"])
feature_vectors["game_half"] = le.fit_transform(feature_vectors["game_half"])
feature_vectors["play_type"] = le.fit_transform(feature_vectors["play_type"])


### Split the data into the training and test sets

In [109]:
# Get the training and test data
train_data, test_data = train_test_split(feature_vectors, test_size=0.2, random_state=35, shuffle=True)

train_labels = train_data[["touchdown"]]
test_labels = test_data[["touchdown"]]

train_data = train_data.drop("touchdown", 1)
test_data = test_data.drop("touchdown", 1)

#print(test_data.iloc[0])

### Set up the KNN model

In [104]:
model = KNeighborsClassifier(n_neighbors=5)

model.fit(train_data, train_labels.values.ravel())
predicted = model.predict(test_data)

# predicted is a ndarray while test_labels is a DataFrame
#print("Predicted: ", predicted[100:150])
#print("Actual Labels: ", test_labels.iloc[100:150])

### Calculate accuracy
Compare predicted with test labels and output the accuracy of our model